# CSV + API

In this reboot, we are going to use:

- The [Goodreads books](https://www.kaggle.com/jealousleopard/goodreadsbooks) dataset from Kaggle.
- The [Open Library Books API](https://openlibrary.org/dev/docs/api/books)

The goal of this livecode is to load the data from a CSV + loop over rows to enrich each row with information such as:

- List of subjects (Science, Humor, Travel, etc.)
- The cover URL of the book
- Other information you'd find useful in the JSON API

First, download the CSV in the local folder:

In [1]:
!curl -L https://gist.githubusercontent.com/ssaunier/351b17f5a7a009808b60aeacd1f4a036/raw/books.csv > books.csv

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 1509k  100 1509k    0     0   793k      0  0:00:01  0:00:01 --:--:--  793k


In [2]:
!ls -lh

total 4344
-rw-r--r--  1 renatoboemer  staff   579B Apr 19 15:07 README.md
-rw-r--r--@ 1 renatoboemer  staff    38K Apr 27 20:09 Recap.ipynb
-rw-r--r--  1 renatoboemer  staff    13K Apr 27 18:12 Recap1.ipynb
-rw-r--r--  1 renatoboemer  staff   1.5M Apr 27 20:17 books.csv


Then import the usual suspects!

In [3]:
# your turn!
import pandas as pd
import requests

In [4]:
# Load our df 
books_df = pd.read_csv('books.csv')
books_df.head()

,bookID,title,authors,average_rating,isbn,isbn13,language_code,# num_pages,ratings_count,text_reviews_count
0,1,Harry Potter and the Half-Blood Prince (Harry ...,J.K. Rowling-Mary GrandPré,4.56,0439785960,9780439785969,eng,652,1944099,26249
1,2,Harry Potter and the Order of the Phoenix (Har...,J.K. Rowling-Mary GrandPré,4.49,0439358078,9780439358071,eng,870,1996446,27613
2,3,Harry Potter and the Sorcerer's Stone (Harry P...,J.K. Rowling-Mary GrandPré,4.47,0439554934,9780439554930,eng,320,5629932,70390
3,4,Harry Potter and the Chamber of Secrets (Harry...,J.K. Rowling,4.41,0439554896,9780439554893,eng,352,6267,272
4,5,Harry Potter and the Prisoner of Azkaban (Harr...,J.K. Rowling-Mary GrandPré,4.55,043965548X,9780439655484,eng,435,2149872,33964


In [5]:
# Let's check for nulls to drop columns / rows
books_df.isnull().sum()

bookID                0
title                 0
authors               0
average_rating        0
isbn                  0
isbn13                0
language_code         0
# num_pages           0
ratings_count         0
text_reviews_count    0
dtype: int64

In [22]:
# Check API and figure out a fetch function
BASE_URL = "https://openlibrary.org/api/books?"

def fetch_book_details(isbn):
    key = f"ISBN:{isbn}"
    param_dict = {'bibkeys': key, "format": 'json', 'jscmd': 'data'}
    response = requests.get(BASE_URL, params=param_dict)
    # we can check our response url for invalid ISBN's
    # looks like we get 200 response but empyt dicts
    # print(response.url)
    if response.json():
        return response.json()[key]
    return None       



In [23]:
# Lets check our function with a valid ISBN and not
print(fetch_book_details('not a book'))
book = fetch_book_details(9780439785969)

None


In [8]:
# retrieve our cover photo from the fetch
def get_cover(book):
    return book.get('cover', {}).get('large')

In [9]:
# Lets check our function with a valid response and not
print(get_cover({}))
get_cover(book)

None


'https://covers.openlibrary.org/b/id/9326654-L.jpg'

In [10]:
# retrieve our subjects from the fetch
def get_subject(book):
    subjects_list = book.get('subjects', [])
    return ', '.join([subject.get('name', None) for subject in subjects_list])

In [11]:
# Lets check our function with a valid response and not
print(get_subject({}))
get_subject(book)

"orphans, foster homes, romans, magie, adolescence, Quill Award winner, Scottish Children’s Book Award winner, British Book of the Year Award winner, Fiction, Juvenile fiction, Magic, Schools, Witches, Wizards, New York Times bestseller, Fantasy fiction, nyt:series_books=2006-07-15, nyt:series_books=2006-09-16, Romans, nouvelles, etc. pour la jeunesse, Sorciers, Roman fantastique, Merveilleux, Hogwarts School of Witchcraft and Wizardry (Imaginary place), Harry Potter (Fictitious character), Hogwarts School of Witchcraft and Wizardry (Imaginary organization), Magos, Magia, Ficción juvenil, Escuelas, Novela fantástica, England, School stories, Family, Harry Potter (Fictional character), Orphans & Foster Homes, Social Themes, Fantasy, Fantasy & Magic, Friendship, Reading Level-Grade 11, Reading Level-Grade 10, Reading Level-Grade 12, England, fiction, Magic, fiction, Schools, fiction, Children's fiction, Potter, harry (fictitious character), fiction, Wizards, fiction, Hogwarts school of w

In [12]:
# retrieve our publisher (limit 1) and publish date from the fetch
# as a tuple
def get_publisher(book):
    publisher = book.get('publishers', [{}])[0].get('name')
    date = book.get('publish_date')
    return (publisher, date)

In [13]:
# Lets check our function with a valid response and not
print(get_publisher({}))
get_publisher(book)

(None, None)


('Scholastic', '2006-09')

In [14]:
# Install the tqdm package so we can see a progress bar on our loop
!pip install tqdm

You should consider upgrading via the '/Users/renatoboemer/.pyenv/versions/3.8.6/envs/lewagon/bin/python3.8 -m pip install --upgrade pip' command.


In [15]:
from tqdm import tqdm

# Use iterrows to loop through our df and add a Cover, Subject, Publisher, Date column
# With our small functions - we can be more dynamic with the columns we want to add
# ex removing "cover" from COLUMNS our loop will still work for all other columns
COLUMNS = ['Cover', 'Publisher', 'Date', 'Subjects']
books_df[COLUMNS] = ''

# wrapping our books_df.iterrows() in a tqdm function gives us a progress bar
# ~1.5 - 2 iterations a second ~2.5 hours to call api for the full dataframe
for index, row in tqdm(books_df.loc[:100,:].iterrows(), total=100):
    book = fetch_book_details(row['isbn13'])
    # Make sure our ISBN worked     
    if book:
        if "Cover" in COLUMNS:
            books_df.loc[index, 'Cover'] = get_cover(book)
        if "Subjects" in COLUMNS:
            # if get_subject returns a list
            # books_df.at[index, 'Subjects'] = get_subject(book)
            # if get_subject returns a string
            books_df.at[index, 'Subjects'] = get_subject(book)
        # if we convert our list to a `set` we can search for a subset (multiple items)         
        if set(['Publisher', 'Date']).issubset(set(COLUMNS)):
            books_df.loc[index, ['Publisher', 'Date']] = get_publisher(book)


101it [01:18,  1.29it/s]                         


In [16]:
books_df[COLUMNS].head(20)

,Cover,Publisher,Date,Subjects
0,https://covers.openlibrary.org/b/id/9326654-L.jpg,Scholastic,2006-09,"orphans, foster homes, romans, magie, adolesce..."
1,https://covers.openlibrary.org/b/id/10716165-L...,Scholastic,2004-09,"Children's Books/Ages 9-12 Fiction, Witches an..."
2,https://covers.openlibrary.org/b/id/7572543-L.jpg,Arthur A. Levine Books,2003,"Ghosts, Monsters, Vampires, Witches, Challenge..."
3,https://covers.openlibrary.org/b/id/10301720-L...,Arthur A. Levine Books,"Nov 01, 2003","Fantasy fiction, school stories, Fiction, Fant..."
4,https://covers.openlibrary.org/b/id/10580458-L...,Bloomsbury,"September 1, 2014","fantasy fiction, orphans, foster homes, fantas..."
5,https://covers.openlibrary.org/b/id/278981-L.jpg,Scholastic Inc.,"October 1, 2004","Potter, harry (fictitious character), fiction,..."
6,https://covers.openlibrary.org/b/id/742235-L.jpg,Nimble Books,"April 26, 2005","Characters, Harry Potter, Children's stories, ..."
7,https://covers.openlibrary.org/b/id/279436-L.jpg,Arthur A. Levine Books,"October 1, 2005","England, fiction, Fantasy fiction, Magic, fict..."
8,https://covers.openlibrary.org/b/id/321859-L.jpg,Gramercy Books,2005,"comic science fiction, Vogons, Humorous fictio..."
9,None,Del Rey,1996,"comic science fiction, Vogons, Humorous fictio..."


## Using Apply Function

In [17]:
def cover_from_apply(row):
    book = fetch_book_details(row['isbn13'])
    if book:
        return book.get('cover', {}).get('large')
    

In [18]:
books_df2 = pd.read_csv('books.csv')

books_df2['Cover']  = ''

In [19]:
%%time
books_df2.loc[:100, 'Cover'] = books_df2.loc[:100,:].apply(lambda x: cover_from_apply(x), axis=1) 

CPU times: user 1.75 s, sys: 124 ms, total: 1.88 s
Wall time: 1min 16s


In [20]:
books_df2.head()

,bookID,title,authors,average_rating,isbn,isbn13,language_code,# num_pages,ratings_count,text_reviews_count,Cover
0,1,Harry Potter and the Half-Blood Prince (Harry ...,J.K. Rowling-Mary GrandPré,4.56,0439785960,9780439785969,eng,652,1944099,26249,https://covers.openlibrary.org/b/id/9326654-L.jpg
1,2,Harry Potter and the Order of the Phoenix (Har...,J.K. Rowling-Mary GrandPré,4.49,0439358078,9780439358071,eng,870,1996446,27613,https://covers.openlibrary.org/b/id/10716165-L...
2,3,Harry Potter and the Sorcerer's Stone (Harry P...,J.K. Rowling-Mary GrandPré,4.47,0439554934,9780439554930,eng,320,5629932,70390,https://covers.openlibrary.org/b/id/7572543-L.jpg
3,4,Harry Potter and the Chamber of Secrets (Harry...,J.K. Rowling,4.41,0439554896,9780439554893,eng,352,6267,272,https://covers.openlibrary.org/b/id/10301720-L...
4,5,Harry Potter and the Prisoner of Azkaban (Harr...,J.K. Rowling-Mary GrandPré,4.55,043965548X,9780439655484,eng,435,2149872,33964,https://covers.openlibrary.org/b/id/10580458-L...
